<a href="https://colab.research.google.com/github/Ellie2499/db_course/blob/main/WEEK13_ASSIGNMENT_12214682.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import sqlite3
import pandas as pd

# Connect to SQLite database
conn = sqlite3.connect('database.db')
c = conn.cursor()

# Create Employees table
c.execute('''CREATE TABLE IF NOT EXISTS Employees (
                ID INTEGER PRIMARY KEY,
                Name TEXT,
                Position TEXT,
                DepartmentID INTEGER,
                Salary INTEGER
            )''')

# Insert sample data into Employees table
sample_employees = [
    (1, 'John Doe', 'Manager', 1, 60000),
    (2, 'Jane Smith', 'Developer', 2, 50000),
    (3, 'Bob Johnson', 'Analyst', 1, 55000),
    (4, 'Alice Brown', 'Designer', 2, 48000),
    (5, 'Charlie Davis', 'Engineer', 1, 65000)
]

c.executemany('INSERT INTO Employees VALUES (?,?,?,?,?)', sample_employees)

# Create Departments table
c.execute('''CREATE TABLE IF NOT EXISTS Departments (
                DepartmentID INTEGER PRIMARY KEY,
                DepartmentName TEXT
            )''')

# Insert sample data into Departments table
sample_departments = [
    (1, 'HR'),
    (2, 'IT'),
    (3, 'Marketing'),
    (4, 'Finance'),
    (5, 'Operations')
]

c.executemany('INSERT INTO Departments VALUES (?,?)', sample_departments)

# Create Projects table
c.execute('''CREATE TABLE IF NOT EXISTS Projects (
                ProjectID INTEGER PRIMARY KEY,
                ProjectName TEXT,
                DepartmentID INTEGER,
                TotalHoursWorked INTEGER DEFAULT 0
            )''')

# Insert sample data into Projects table
sample_projects = [
    (101, 'Project A', 1, 0),
    (102, 'Project B', 2, 0),
    (103, 'Project C', 3, 0),
    (104, 'Project D', 1, 0),
    (105, 'Project E', 2, 0)
]

c.executemany('INSERT INTO Projects VALUES (?,?,?,?)', sample_projects)


# Create Assignments table
c.execute('''CREATE TABLE IF NOT EXISTS Assignments (
                EmployeeID INTEGER,
                ProjectID INTEGER,
                HoursWorked INTEGER,
                PRIMARY KEY (EmployeeID, ProjectID),
                FOREIGN KEY (EmployeeID) REFERENCES Employees(ID),
                FOREIGN KEY (ProjectID) REFERENCES Projects(ProjectID)
            )''')

# Create the trigger to update total hours worked in Projects table
c.execute('''
    CREATE TRIGGER IF NOT EXISTS update_total_hours
    AFTER INSERT ON Assignments
    BEGIN
      UPDATE Projects
      SET TotalHoursWorked = TotalHoursWorked + NEW.HoursWorked
      WHERE ProjectID = NEW.ProjectID;
    END;
''')

# Commit the changes and close the connection
conn.commit()
conn.close()




In [4]:
import sqlite3
import pandas as pd

# Connect to SQLite database using the with statement
with sqlite3.connect('database.db') as conn:
    # Create a cursor
    c = conn.cursor()

    # Execute the query to list all employees along with the names of the projects they are working on
    join_query = """
    SELECT
      Employees.Name AS EmployeeName,
      Employees.Position,
      Projects.ProjectName
    FROM
      Employees
    JOIN
      Assignments ON Employees.ID = Assignments.EmployeeID
    JOIN
      Projects ON Assignments.ProjectID = Projects.ProjectID
    """

    # Fetch and print the results of the join query
    c.execute(join_query)
    join_results = c.fetchall()
    print("Join results:", join_results)

    # Define the subquery to find the department with the highest average salary
    subquery = """
    WITH AverageSalary_CTE AS (
      SELECT
        DepartmentID,
        AVG(Salary) AS AvgSalary
      FROM
        Employees
      GROUP BY
        DepartmentID
      ORDER BY
        AvgSalary DESC
      LIMIT 1
    )
    SELECT
      DepartmentID,
      AvgSalary
    FROM
      AverageSalary_CTE
    """

    # Execute the subquery and read the results into a DataFrame
    average_salary_df = pd.read_sql_query(subquery, conn)

    # Print the results of the average salary subquery
    print("Average Salary:")
    print(average_salary_df)

    # Define the ranking query using a window function
    ranking_query = """
    SELECT
      ID AS EmployeeID,
      Name,
      Position,
      DepartmentID,
      Salary,
      RANK() OVER (PARTITION BY DepartmentID ORDER BY Salary DESC) AS Rank
    FROM
      Employees
    """

    # Execute the ranking query and read the results into a DataFrame
    ranking_df = pd.read_sql_query(ranking_query, conn)

    # Print the results of the ranking query
    print("Ranking results:")
    print(ranking_df)


Join results: []
Average Salary:
   DepartmentID  AvgSalary
0             1    60000.0
Ranking results:
   EmployeeID           Name   Position  DepartmentID  Salary  Rank
0           5  Charlie Davis   Engineer             1   65000     1
1           1       John Doe    Manager             1   60000     2
2           3    Bob Johnson    Analyst             1   55000     3
3           2     Jane Smith  Developer             2   50000     1
4           4    Alice Brown   Designer             2   48000     2
